In [38]:
!pip install gradio
import gradio as gr

In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import gradio as gr

In [40]:
!wget https://gist.githubusercontent.com/Momnadar1/8805a6d53e92d6be17b9837711a5931a/raw/adc9cc97efc92232f01cbb6a1b13e8123d9f8f8d/shakepeare_s_plays.txt

--2024-11-05 14:44:27--  https://gist.githubusercontent.com/Momnadar1/8805a6d53e92d6be17b9837711a5931a/raw/adc9cc97efc92232f01cbb6a1b13e8123d9f8f8d/shakepeare_s_plays.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5583374 (5.3M) [text/plain]
Saving to: ‘shakepeare_s_plays.txt.1’

shakepeare_s_plays. 100%[===================>]   5.32M  --.-KB/s    in 0.09s   

2024-11-05 14:44:27 (60.3 MB/s) - ‘shakepeare_s_plays.txt.1’ saved [5583374/5583374]



In [41]:
with open('shakepeare_s_plays.txt', 'r') as f:
    text = f.read()
# print from the data
print(text[:100])

# Hamlet

ACT I
SCENE I. Elsinore. A platform before the castle.

    FRANCISCO at his post. Enter t


In [42]:
# data preprocessing
# mapping each character with it's index
chars = sorted(list(set(text))) # chars represent special characters
vocab_size = len(chars) # getting the len of distinct characters
print(''.join(chars), vocab_size)

	
 !#$&'(),-.0123456789:;=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyzÀè—‘’“”… 90


In [43]:
# encoding and decoding function for training and testing the machine learning models
str_to_int = {char:i for i, char in enumerate(chars)}
int_to_str = {i:char for i, char in enumerate(chars)}

encode = lambda string: [str_to_int[s] for s in string]
decode = lambda indexes: [int_to_str[i] for i in indexes]

''.join(decode(encode('hi work!!')))

'hi work!!'

In [44]:
str_to_int # mapping each character with it's index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '#': 4,
 '$': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22,
 ':': 23,
 ';': 24,
 '=': 25,
 '>': 26,
 '?': 27,
 'A': 28,
 'B': 29,
 'C': 30,
 'D': 31,
 'E': 32,
 'F': 33,
 'G': 34,
 'H': 35,
 'I': 36,
 'J': 37,
 'K': 38,
 'L': 39,
 'M': 40,
 'N': 41,
 'O': 42,
 'P': 43,
 'Q': 44,
 'R': 45,
 'S': 46,
 'T': 47,
 'U': 48,
 'V': 49,
 'W': 50,
 'X': 51,
 'Y': 52,
 'Z': 53,
 '[': 54,
 ']': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 'À': 82,
 'è': 83,
 '—': 84,
 '‘': 85,
 '’': 86,
 '“': 87,
 '”': 88,
 '…': 89}

In [45]:
data = torch.tensor(encode(text), dtype=torch.long)
data[:100], data.shape, data.dtype

(tensor([ 4,  2, 35, 56, 68, 67, 60, 75,  1,  1, 28, 30, 47,  2, 36,  1, 46, 30,
         32, 41, 32,  2, 36, 12,  2, 32, 67, 74, 64, 69, 70, 73, 60, 12,  2, 28,
          2, 71, 67, 56, 75, 61, 70, 73, 68,  2, 57, 60, 61, 70, 73, 60,  2, 75,
         63, 60,  2, 58, 56, 74, 75, 67, 60, 12,  1,  1,  2,  2,  2,  2, 33, 45,
         28, 41, 30, 36, 46, 30, 42,  2, 56, 75,  2, 63, 64, 74,  2, 71, 70, 74,
         75, 12,  2, 32, 69, 75, 60, 73,  2, 75]),
 torch.Size([5580526]),
 torch.int64)

In [46]:
splitter = int(0.9*len(data)) # calculate the number which represent 90% of the data

train, test = data[:splitter], data[splitter:] #train from 0 to 90 and the test from 90 to 100

In [47]:
n_emb = 64
block_size = 32
# head_size = 4
n_x = 4
num_heads = 4
eval_iteration = 250
max_iters = 5000
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [48]:
#%%
class Head(nn.Module):
    """
    one head in self attention
    """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size)
        self.query = nn.Linear(n_emb, head_size)
        self.value = nn.Linear(n_emb, head_size)
        self.dropout = nn.Dropout(0.0)

        # tril: lower-triangular
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        batch, blocks, X = x.shape
        query = self.query(x) # batch, block_size, X -- shape
        key = self.key(x) # batch, block_size, X -- shape
        weight = query @ key.transpose(-2, -1)  * X ** -0.5 # batch, block_size, X @ batch, X, blocl_size ---> batch, block_size, block_size
        weight = weight.masked_fill(self.tril[:blocks, :blocks] == 0,float('-inf'))
        weight = F.softmax(weight, dim=-1)
        weight = self.dropout(weight)
        out = weight @  self.value(x)
        return out

class MultiHeadAttention(nn.Module):
    """
    multi head in self attention
    """
    # nnum_head = 6
    # head_size
    def __init__(self, head_size, num_heads):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.layer = nn.Linear(n_emb, n_emb)
        self.dropout = nn.Dropout(0.0)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.layer(out))


class FeedForward(nn.Module):

    def __init__(self, n_emb):
        super().__init__()
        self.dff = nn.Sequential(
            nn.Linear(n_emb, n_emb*4),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb),
            nn.Dropout(0.0)
        )

    def forward(self, x):
        return self.dff(x)

class BlockSeq(nn.Module):
    def __init__(self, n_emb, num_heads):
        super().__init__()
        head_size = int(n_emb / num_heads)
        self.mh_att = MultiHeadAttention(head_size, num_heads)
        self.ff_lay = FeedForward(n_emb)
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self, x):
        x = x + self.mh_att(self.ln1(x))
        x = x + self.ff_lay(self.ln2(x))
        return x


class TextGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        #  x = [1, 25, 89, 65,63,64]
        self.lookup_token_emd_table = nn.Embedding(vocab_size, n_emb)
        self.postional_encoding = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[BlockSeq(n_emb, num_heads) for _ in range(n_x)])
        self.layer_norm = nn.LayerNorm(n_emb)
        self.model_head = nn.Linear(n_emb, vocab_size)

    def forward(self, x, y=None): #
        batches, block_size_x = x.shape
        out = self.lookup_token_emd_table(x) # 2, 7, 90 , x: 1,2 3
        pos_enc = self.postional_encoding(torch.arange(block_size_x, device=device))
        out = out + pos_enc
        out = self.blocks(out)
        out = self.layer_norm(out)
        out = self.model_head(out)


        if y is None:
            loss = None
        else:
            batches, block_size, X = out.shape
            loss = F.cross_entropy(out.view(batches*block_size, X), y.view(batches*block_size))

        return out, loss

    def generate(self, x, max_tokens=200):
        for _ in range(max_tokens):
            logits, _ = self(x[:, -block_size:])
            logits = logits[:, -1, :]
            # print(logits.shape)
            probilities = F.softmax(logits, dim=-1) # 1, 90
            next_x = torch.multinomial(probilities, num_samples=1)
            x = torch.cat((x, next_x), dim=1) # [hi, ] 1 2 3
        return x


In [49]:
def batches(split):
    data = train if split == 'train' else test
    #
    indexes = torch.randint(len(data) - block_size, (batch_size, ))
    # print(indexes)
    x = torch.stack([data[i:i+block_size] for i in indexes])
    y = torch.stack([data[i+1:i+block_size+1] for i in indexes])
        # data[4020066: 4020066+block]
    x, y = x.to(device), y.to(device)
    return x, y


In [50]:
@torch.no_grad()
def losses_estimate():
  output = {}
  model.eval()
  for split in ['train', 'test']:
    losses = torch.zeros(eval_iteration)
    for i in range(eval_iteration):
      x, y = batches(split)
      logits, loss = model(x, y)
      losses[i] = loss.item()
    output[split] = losses.mean()
  model.train()
  return output

In [51]:
meta = {
    'vocab_size': vocab_size,
    'itos': int_to_str,
    'stoi': str_to_int,
}
with open('meta.pkl', 'wb') as f:
    pickle.dump(meta, f)

In [52]:
model = TextGenerator()
model = model.to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'Millions parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=6e-4)
for iter in range(max_iters):
  if iter % eval_iteration == 0 :
    losses = losses_estimate()
    print(f"Iteration no: {iter}, training loss: {losses['train']:.3f}  and val loss: {losses['test']:.3f}")

  x, y = batches('train')
  logits, loss = model(x, y)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

0.213722 Millions parameters
Iteration no: 0, training loss: 4.562  and val loss: 4.583
Iteration no: 250, training loss: 2.371  and val loss: 2.547
Iteration no: 500, training loss: 2.182  and val loss: 2.406
Iteration no: 750, training loss: 2.072  and val loss: 2.323
Iteration no: 1000, training loss: 1.983  and val loss: 2.242
Iteration no: 1250, training loss: 1.931  and val loss: 2.202
Iteration no: 1500, training loss: 1.884  and val loss: 2.148
Iteration no: 1750, training loss: 1.845  and val loss: 2.129
Iteration no: 2000, training loss: 1.818  and val loss: 2.109
Iteration no: 2250, training loss: 1.798  and val loss: 2.087
Iteration no: 2500, training loss: 1.763  and val loss: 2.049
Iteration no: 2750, training loss: 1.752  and val loss: 2.050
Iteration no: 3000, training loss: 1.727  and val loss: 2.027
Iteration no: 3250, training loss: 1.707  and val loss: 2.014
Iteration no: 3500, training loss: 1.704  and val loss: 2.011
Iteration no: 3750, training loss: 1.690  and v

In [53]:
model = torch.save(model, 'entire_model.pth') # save the model

In [54]:
with open('meta.pkl', 'rb') as f:
    meta = pickle.load(f)
stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: [itos[i] for i in l]

In [55]:
def reply(message, history):
    # encode the beginning of the prompt
    start = message
    start_ids = encode(start)
    x = (torch.tensor(start_ids, dtype=torch.long, device='cuda')[None, ...])
    print(x)
    replied = []
    # run generation
    with torch.no_grad():
        for k in range(3):
            y = model.generate(x, 20)
            replied.append(''.join(decode(y[0].tolist())))
    return '\n'.join(replied)

In [60]:
from gradio_client import Client

client = Client("https://4072dbe68514fc9d1d.gradio.live/")
try:
    result = client.predict(message="Hello!!", api_name="/predict")
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")

Loaded as API: https://4072dbe68514fc9d1d.gradio.live/ ✔


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2015, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1562, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

An error occurred: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


In [64]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bfc96709b29198af37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [65]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="textbox", outputs="textbox")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab10d7bab6f4ab53c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
